# PuLP Tutorial

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pulp

In [4]:
n_warehouses = 2
n_customers = 4

# Cost Matrix
cost_matrix = np.array([
    [1, 3, 0.5, 4],
    [2.5, 5, 1.5, 2.5]
])
# Demand Matrix
cust_demands = np.array([35000, 22000, 18000, 30000])

# Supply Matrix
warehouse_supply = np.array([60000, 80000])

In [6]:
model = pulp.LpProblem("Supply-Demand-Problem", pulp.LpMinimize)

In [8]:
# Decision variables
var_names = [f"{i}{j}" for i in range(1, n_warehouses+1) for j in range(1, n_customers+1) ]
var_names

['11', '12', '13', '14', '21', '22', '23', '24']

In [9]:
decision_vars = pulp.LpVariable.matrix('X', var_names, cat='Integer', lowBound=0)
allocation = np.array(decision_vars).reshape(2,4)
print('Decision Variable/Allocadtion Matrix: ')
print(allocation)

Decision Variable/Allocadtion Matrix: 
[[X_11 X_12 X_13 X_14]
 [X_21 X_22 X_23 X_24]]


In [14]:
obj_func = pulp.lpSum(allocation * cost_matrix)
print(obj_func)

X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24


In [16]:
model += obj_func
print(model)

Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer



In [25]:
# Supply constraints

# for i in range(n_warehouses):
for i in range(n_warehouses):
    print(pulp.lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i])
    model += pulp.lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i], f"Supply Constraints {i}"


X_11 + X_12 + X_13 + X_14 <= 60000
X_21 + X_22 + X_23 + X_24 <= 80000


In [28]:
# Customer demand constraints

for j in range(n_customers):
    print(pulp.lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j])
    model += pulp.lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[i], f"Demand Constraints {j}"

X_11 + X_21 >= 35000
X_12 + X_22 >= 22000
X_13 + X_23 >= 18000
X_14 + X_24 >= 30000


In [29]:
model

Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
SUBJECT TO
Supply_Constraints_0: X_11 + X_12 + X_13 + X_14 <= 60000

Supply_Constraints_1: X_21 + X_22 + X_23 + X_24 <= 80000

Demand_Constraints_0: X_11 + X_21 >= 22000

Demand_Constraints_1: X_12 + X_22 >= 22000

Demand_Constraints_2: X_13 + X_23 >= 22000

Demand_Constraints_3: X_14 + X_24 >= 22000

VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer

In [30]:
model.writeLP('Supply_demand_prob.lp')

[X_11, X_12, X_13, X_14, X_21, X_22, X_23, X_24]

In [35]:
# model.solve(pulp.PULP_CBC_CMD())
model.solve()
status = pulp.LpStatus[model.status]
print(status)

Optimal


In [36]:
model.objective.value()

160000.0

In [38]:
for v in model.variables():
    try:
        print(v.name, '=', v.value())
    except:
        print('error couldn\'t find value')

X_11 = 22000
X_12 = 22000
X_13 = 16000
X_14 = 0
X_21 = 0
X_22 = 0
X_23 = 6000
X_24 = 22000


In [39]:
pulp.pulpTestAll()

ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss................

	 Test that logic put in place for deprecation handling of indexs works
	 Testing 'indexs' param continues to work for LpVariable.dicts
	 Testing 'indexs' param continues to work for LpVariable.matrix
	 Testing 'indices' argument works in LpVariable.dicts
	 Testing 'indices' param continues to work for LpVariable.matrix
	 Testing invalid status
	 Testing continuous LP solution - export dict
	 Testing export dict for LP
	 Testing export dict MIP
	 Testing maximize continuous LP solution
	 Testing continuous LP solution - export JSON
	 Testing continuous LP solution - export solver dict
	 Testing continuous LP solution - export solver JSON
	 Testing reading MPS files - binary variable, no constraint names
	 Testing reading MPS files - integer variable
	 Testing reading MPS files - maximize


................

	 Testing invalid var names
	 Testing makeDict general behavior
	 Testing makeDict default value behavior
	 Testing measuring optimization time
	 Testing the availability of the function pulpTestAll
	 Testing zero subtraction
	 Testing inconsistent lp solution
	 Testing continuous LP solution
	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint
	 Testing zero objective
	 Testing LpVariable (not LpAffineExpression) objective


.........

...

...........

	 Testing an integer infeasible problem
	 Testing another integer infeasible problem
	 Testing column based modelling
	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)


....sssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss

	 Testing elastic constraints (penalty unbounded)
	 Testing timeLimit argument


sssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss
----------------------------------------------------------------------
Ran 840 tests in 2.031s

OK (skipped=784)


In [40]:
pulp.listSolvers(onlyAvailable=True)

['GLPK_CMD']

# Scipy Tutorial

## Minimize

$ min z = 10x_1 + 15x_2 + 25x_3\\
s.t.\\
1x_1 + 1x_2 + 1x_3 \geq 1000\\
1x_1 - 2x_2 + 0x_3 \geq 0\\
0x_1 + 0x_2 + 1x_3 \geq 340\\
x_1, x_2, x_3 \geq 0
$

In [46]:
import numpy as np
from scipy.optimize import linprog

# Set the inequality constraints matrix
# Note: the inequality constraints must be in the form of <=

A = np.array([[-1, -1, -1], 
              [-1, 2, 0],
              [0, 0, -1],
              [-1, 0, 0],
              [0, -1, 0],
              [0, 0, -1]])

# Set the inequality constraints vector
b = np.array([-1000, 0, -340, 0, 0, 0])

# Set the coefficients of the linear objective function vector
c = np.array([10, 15, 25])

# Solve linear programming problem
res = linprog(c, A_ub=A, b_ub=b)

# Print results
print('Optimal value:', round(res.fun, ndigits=2),
      '\nx values:', res.x,
      '\nNumber of iterations performed:', res.nit,
      '\nStatus:', res.message)

Optimal value: 15100.0 
x values: [6.59999996e+02 1.00009441e-07 3.40000000e+02] 
Number of iterations performed: 7 
Status: Optimization terminated successfully.


## Maximize

$max \quad  z = 5x_1 + 7x_2\\
s.t.\\
1x_1 + 0x_2 \leq 16\\
2x_1 + 3x_2 \leq 19\\
1x_1 + 1x_2 \leq 8\\
x_1, x_2 \geq 0
$

In [48]:
A = np.array([
    [1, 0],
    [2, 3],
    [1, 1],
    [-1, 0],
    [0, -1]
])
b = np.array([16, 19, 8, 0, 0])
c = np.array([-5, -7])
res = linprog(c, A_ub=A, b_ub=b)


print('Optimal value:', round(res.fun, ndigits=2),
      '\nx values:', res.x,
      '\nNumber of iterations performed:', res.nit,
      '\nStatus:', res.message)

Optimal value: -46.0 
x values: [5. 3.] 
Number of iterations performed: 5 
Status: Optimization terminated successfully.


## Implement the above example from PuLP

* Can't solve integer programming

$Minimize \quad \sum_{1=1}^{2} \sum_{j=1}^{4} c_{ij} x_{ij}\\
s.t.\\
\sum_{i=1}^{2} x_{i1} \geq 35000\\
\sum_{i=1}^{2} x_{i2} \geq 22000\\
\sum_{i=1}^{2} x_{i3} \geq 18000\\
\sum_{i=1}^{2} x_{i4} \geq 30000\\
\\
\sum_{j=1}^{4} x_{1j} \leq 60000\\
\sum_{j=1}^{4} x_{2j} \leq 80000\\
\\
x_{ij} \geq 0
$

In [50]:
A = np.array([
    [-1, 0, 0, 0,-1, 0, 0, 0],
    [ 0,-1, 0, 0, 0,-1, 0, 0],
    [ 0, 0,-1, 0, 0, 0,-1, 0],
    [ 0, 0, 0,-1, 0, 0, 0,-1],
    [ 1, 1, 1, 1, 0, 0, 0, 0],
    [ 0, 0, 0, 0, 1, 1, 1, 1],
    [-1, 0, 0, 0, 0, 0, 0, 0],
    [ 0,-1, 0, 0, 0, 0, 0, 0],
    [ 0, 0,-1, 0, 0, 0, 0, 0],
    [ 0, 0, 0,-1, 0, 0, 0, 0],
    [ 0, 0, 0, 0,-1, 0, 0, 0],
    [ 0, 0, 0, 0, 0,-1, 0, 0],
    [ 0, 0, 0, 0, 0, 0,-1, 0],
    [ 0, 0, 0, 0, 0, 0, 0,-1],
])
b = np.array([-35000, -22000, -18000, -30000, 60000, 80000,
              0, 0, 0, 0, 0, 0, 0, 0])
c = np.array([1, 1, 1, 1, 1, 1, 1, 1])

res = linprog(c, A_ub=A, b_ub=b)

# Print results
print('Optimal value:', round(res.fun, ndigits=2),
      '\nx values:', res.x,
      '\nNumber of iterations performed:', res.nit,
      '\nStatus:', res.message)

Optimal value: 105000.0 
x values: [16385.65740888  9728.93801128  8805.88121169 13206.81319763
 18614.34252826 12271.06194166  9194.11874856 16793.18673622] 
Number of iterations performed: 7 
Status: Optimization terminated successfully.


In [58]:

labor_cost_rates = np.array([[11, 15]])

labor_hours = np.array([[5, 1], [6, 2], [8, 3]])

labor_cost_rates @ labor_hours.T

array([[ 70,  96, 133]])

In [60]:
cost_parts = np.array([[150, 225, 275]])
selling_prices = np.array([[300, 450, 560]])

margins = selling_prices - cost_parts - labor_cost_rates @ labor_hours.T
margins

array([[ 80, 129, 152]])

$ Maximize \sum_{i=1}^{3} m_i x_i\\
s.t.\\
x_1 \leq 600\\
x_2 \leq 1200\\
x_3 \leq 50\\
5x_1 + 6x_2 + 8x_3 \leq 10000\\
1x_1 + 2x_2 + 3x_3 \leq 3000\\
x_1, x_2, x_3 \geq 0
$

In [69]:
A = np.array([
    [ 1, 0, 0],
    [ 0, 1, 0],
    [ 0, 0, 1],
    [ 5, 6, 8],
    [ 1, 2, 3],
    [-1, 0, 0],
    [ 0,-1, 0],
    [ 0, 0, -1]
])
b = np.array([600, 1200, 50, 10000, 3000, 0, 0, 0])
c = np.array([-80, -129, -152])

res = linprog(c, A_ub=A, b_ub=b)
print('Optimal value:', round(-res.fun, ndigits=2),
      '\nx values:', res.x,
      '\nNumber of iterations performed:', res.nit,
      '\nStatus:', res.message)

Optimal value: 200285.71 
x values: [ 514.28571289 1199.99999645   28.57142846] 
Number of iterations performed: 7 
Status: Optimization terminated successfully.


In [130]:
from pulp import *



sku = ['Basic', 'XP', 'VXP']
n_sku = 3       
margins = np.array([80, 129, 152])
       

# margins = {
#     'Basic' : 80,
#     'XP' : 129,
#     'VXP' : 152
# }

prob = LpProblem('ProductMix', LpMaximize)
decision_vars = LpVariable.matrix('X', range(1, len(sku)+1), 
                                  cat='Continuous',#'Integer',
                                  lowBound=0)
obj_func = lpSum(decision_vars * margins)
prob += obj_func
print(obj_func)

80*X_1 + 129*X_2 + 152*X_3


In [131]:
# Product constraints
sku_limits = np.array([600, 1200, 50])
for i in range(n_sku):
    print(decision_vars[i] <= sku_limits[i])
    prob += (decision_vars[i] <= sku_limits[i]), f'SKU Constraints {i+1}'
prob

X_1 <= 600
X_2 <= 1200
X_3 <= 50


ProductMix:
MAXIMIZE
80*X_1 + 129*X_2 + 152*X_3 + 0
SUBJECT TO
SKU_Constraints_1: X_1 <= 600

SKU_Constraints_2: X_2 <= 1200

SKU_Constraints_3: X_3 <= 50

VARIABLES
X_1 Continuous
X_2 Continuous
X_3 Continuous

In [134]:
for i in range(len(labor_limits)):
    print(lpSum(decision_vars * labor_cost_hrs[i]) <= labor_limits[i])
    prob += lpSum(decision_vars * labor_cost_hrs[i]) <= labor_limits[i]
prob    

5*X_1 + 6*X_2 + 8*X_3 <= 10000
X_1 + 2*X_2 + 3*X_3 <= 3000


ProductMix:
MAXIMIZE
80*X_1 + 129*X_2 + 152*X_3 + 0
SUBJECT TO
SKU_Constraints_1: X_1 <= 600

SKU_Constraints_2: X_2 <= 1200

SKU_Constraints_3: X_3 <= 50

_C1: 5 X_1 + 6 X_2 + 8 X_3 <= 10000

_C2: X_1 + 2 X_2 + 3 X_3 <= 3000

VARIABLES
X_1 Continuous
X_2 Continuous
X_3 Continuous

In [136]:
prob.solve()
print(LpStatus[prob.status])
print('Objective value:', prob.objective.value())
for v in prob.variables():
    print(v.name,'=', v.value())

Optimal
Objective value: 200285.7328
X_1 = 514.286
X_2 = 1200.0
X_3 = 28.5714
